In [ ]:
import numpy as np
import tensorflow as tf
from skimage.transform import resize

def load_test_data():
    data = np.load('data_test.npy')  # Replace with your test data file
    labels = np.load('labels_test.npy')  # Replace with your test labels file

    # Replicating the same preprocessing as in the training script
    data_reshaped = data.T.reshape((-1, 300, 300, 3))
    resized_data = np.array([resize(img, (100, 100, 3), anti_aliasing=True) for img in data_reshaped])
    
    if resized_data.max() > 1.0:
        data_normalized = resized_data.astype(np.float32) / 255.0
    else:
        data_normalized = resized_data

    return data_normalized, labels

def test(model, test_data, test_labels, confidence_threshold=0.5):
    loss, accuracy = model.evaluate(test_data, test_labels)
    predictions = model.predict(test_data)
    
    # Compute confidence scores (max probability)
    confidence_scores = np.max(predictions, axis=1)

    # Assign -1 to predictions with confidence lower than threshold
    predicted_labels = np.where(confidence_scores < confidence_threshold, -1, np.argmax(predictions, axis=1))

    # Count the number of unknown predictions
    unknown_count = np.sum(predicted_labels == -1)

    return accuracy, predicted_labels, unknown_count

if __name__ == "__main__":
    test_data, test_labels = load_test_data()
    model = tf.keras.models.load_model('final_model.h5')

    # Set a confidence threshold
    confidence_threshold = 0.5  # Adjust based on validation

    accuracy, predicted_labels, unknown_count = test(model, test_data, test_labels, confidence_threshold)

    print(f"Test Accuracy: {accuracy}")
    print(f"Number of Unknown Images Classified: {unknown_count}")

    # Print all predicted labels side by side
    print("All Predicted Labels:")
    print(' , '.join(map(str, predicted_labels)))
    
    # Option 2: Save the predicted labels to a file
    # np.savetxt("predicted_labels.txt", predicted_labels, fmt="%d")
